In [52]:
import topogenesis as tg
import numpy as np
np.random.seed(0)

In [53]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of walking method for agents
    def walk(self, env):
        # find available spaces
        #######################

        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        # retrieve the myvalue of each neighbour
        free_neighs_myvalue = env.myvalue.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_myvalue)]
        
        # update information
        ####################

        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()

In [54]:
# initiate availibility lattice
unit = 1
bounds = np.array([[0,0,0],[0,8,8]])
avail_lattice = tg.lattice(bounds, unit=unit, default_value=1, dtype=int)

# randomly scattering the agents
selected_cells = np.random.choice(avail_lattice.size, 5)
agent_ind = np.array(np.unravel_index(selected_cells, avail_lattice.shape))

# creating neighborhood definition
stencil = tg.stencil(np.array([[[0, 1, 0,],
                                [1, 1, 1,],
                                [0, 1, 0,]]]),origin=np.array([0,1,1]))

agents= []
# creating agent objects
for id, ind in enumerate(agent_ind.T.tolist()):
    myagent = agent(ind, stencil, id+1)
    agents.append(myagent)

In [55]:
# environment class
class environment():
    def __init__(self, lattices, agents):
        self.availibility = lattices["availibility"]
        self.myvalue = lattices["myvalue"]
        self.agent_origin = self.availibility * 0
        self.agents = agents
        self.update_agents()
    
    def update_agents(self):
        for a in self.agents:
            # making previous position available
            self.availibility[tuple(a.old_origin)] = self.availibility[tuple(a.old_origin)] * 0 + 1
            # removing agent from previous position
            self.agent_origin[tuple(a.old_origin)] *= 0
            # making the current position unavailable
            self.availibility[tuple(a.origin)] *= 0
            # adding agent to the new position 
            self.agent_origin[tuple(a.origin)] = a.id
    
    def walk_agents(self):
        # iterate over egents and perform the walk
        for a in self.agents:
            a.walk(self)
        # update the agent states in environment
        self.update_agents()

In [56]:
# construct a dummy value field
###############################

# create a series of sin values for 0 to pi
sin_a = np.sin(np.arange(9) * 0.125 * np.pi).astype(np.float16)
# compute the outer product of the series with itself to create a radial value field
myvalue_field = np.outer(sin_a,sin_a)
# add extra dimension to array to make it comaptible with lattices
myvalue_field = myvalue_field[np.newaxis, :]
# construct the lattice
myvalue_lattice = tg.to_lattice(myvalue_field, np.array([0,0,0]))

In [57]:
# initiate the environment
env_lattices = {"availibility": avail_lattice,
                "myvalue": myvalue_lattice}
env = environment(env_lattices, agents)
for i in range(10):
    # print(env.availibility)
    # print(env.agent_origin)
    agn_org = [a.origin for a in env.agents]
    print(agn_org)
    env.walk_agents()

[array([0, 4, 8]), array([0, 5, 2]), array([0, 7, 1]), array([0, 7, 4]), array([0, 7, 4])]
[array([0, 4, 7]), array([0, 5, 3]), array([0, 6, 1]), array([0, 6, 4]), array([0, 6, 4])]
[array([0, 4, 6]), array([0, 4, 3]), array([0, 6, 2]), array([0, 5, 4]), array([0, 5, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 5, 2]), array([0, 4, 4]), array([0, 4, 4])]
[array([0, 3, 5]), array([0, 3, 4]), array([0, 5, 3]), array([0, 3, 4]), array([0, 3, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 4, 3]), array([0, 4, 4]), array([0, 4, 4])]
[array([0, 3, 5]), array([0, 3, 4]), array([0, 3, 3]), array([0, 3, 4]), array([0, 3, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 4, 3]), array([0, 4, 4]), array([0, 4, 4])]
[array([0, 3, 5]), array([0, 3, 4]), array([0, 3, 3]), array([0, 3, 4]), array([0, 3, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 4, 3]), array([0, 4, 4]), array([0, 4, 4])]
